### Navigation Reminder

- **Grey cells** are **code cells**. Click inside them and type to edit.
- **Run**  code cells by pressing $ \triangleright $  in the toolbar above, or press ``` shift + enter```.
-  **Stop** a running process by clicking $\Box$ in the toolbar above.
- You can **add new cells** by clicking to the left of a cell and pressing ```A``` (for above), or```B``` (for below). 
- **Delete cells** by pressing```X```.
- Run all code cells that import objects (such as the one below) to ensure that you can follow exercises and examples.
- Feel free to edit and experiment - you will not corrupt the original files.

# Pandas II: Making the Most of Tabular Data

In the previous lesson, we learned how to access data contained within a DataFrame. Next, we learn to clean, restructure, combine, analyze and visualize tabular information with Pandas.

---
Questions and exercises are distributed throughout this lesson. Please run the code cell below to import them before starting the lesson. The code will not produce any visible output, but they will be loaded for future use.

In [ ]:
from QuestionsPandas2 import Q3, question, solution

---
## Lesson Goals

- Sort, add and remove columns and rows
- Perform math on columns and rows
- Understand Tidy Data and pivot using **df.melt()**
- Join and append tables through **df.merge()** and **df.append()**
- Understand basics of **visualizing dataframes** in Pandas

**Key Concepts:** Tidy data, join, merge, melt

---

Let's go ahead and import Pandas as pd and create a DataFrame called object_table with the same excel file we used then. The file path is 'Other_files/NGAData.xlsx', and the sheet_name is 'objects'. Try it out for yourself.

In [ ]:
objects_table.set_index('objectid',inplace=True)

In [ ]:
objects_table.columns

In [ ]:
solution(Q1)

# A Note on Tidy Data

**Tidy**, or **tall**, **data** is the preferred format for use with Pandas. If you are familiar with R or Tableau, you should be familiar with tidy data.

Datasets contain **values**, **variables** and **observations**.

>A dataset is a collection of **values**, usually either numbers (if quantitative) or strings (if qualitative). Values are organised in two ways. Every value belongs to a variable and an observation. A **variable** contains all values that measure the same underlying attribute (like height, temperature, duration) across units. An **observation** contains all values measured on the same unit (like a person, or a day, or a race) across attributes.

**With tidy data, each variable forms one column, each observation forms a row, and each observational unit forms a table.** 

Often, datasets don't fulfill these requisites, with common problems being that:

- Column headers are values, not variable names (i.e., one variable is stored in several columns).
- Multiple variables are stored in one column.
- Variables are stored in both rows and columns.
- Multiple types of observational units are stored in the same table. 
- A single observational unit is stored in multiple tables.

Visually, a dataset like this:

| person | treatment | result |
|--|--|--|
|John|a|NaN|
|Jane|a|16|
|Mary|a|3|
|John|b|2|
|Jane|b|11|
|Mary|b|1|

Is better than one like this:

| person | treatment a| treatment b |
|--|--|--|
|John|NaN|2|
|Jane|16|11|
|Mary|3|1|

That is because the dataset contains 18 values representing 3 variables and 6 observations
- person, with three possible values (John, Mary and Jane)
- treatment, with two possible values (a and b)
- result, with five or six possible values, depending on how you think of the missing value (-, 16, 3, 2, 11, 1)

Source:http://vita.had.co.nz/papers/tidy-data.pdf

We won't get further into tidy data here, but a great resource are Eric Monson's talks below, and his Github repository on Jupyter and Pandas, which includes two notebooks on Tidy Data.


# Editing & Cleaning DataFrames

Oftentimes, we will need to edit the columns and rows in a table in different ways. In this section, we will look at sorting dataframes, adding and removing columns, joining datasets and removing duplicate values, as well as simple math functions.  

## Sorting Dataframes

```df.sort_values()``` is the method for sorting dataframes according to a column's contents. When applying this, we must note that it is not affecting the original dataframe. Thus, we either have to assign our modified dataframe to a variable or specify that we want the method to modify the original values.

We should be familiar with assigning variables by now:

```python
df2= df2.sort_values('column') 
```

In this case, we are reassigning the original dataframe variable to its new value, a common approach.

Alternatively, through the in_place keyword, we can modify the dataframe directly.

```python
df2.sort_values('column', in_place=True) 
```

The default is to sort by ascending order; to change to descending, we must specify ascending=False.

In our object table, it might make sense to sort objects by their year of creation.

In [ ]:
objects_table.sort_values('beginyear')

## Adding and Removing columns

To add a column, just call an unassigned column name with bracket notation and specify the desired value for all cells.

```python
df['newname'] = x
```

To remove a column, use the df.drop() method:

```python
df = df.drop('column_name', 1)
```
Once again, you have to reassign the variable or specify in_place=True.
You must also specify the axis: 1 refers to columns, 0 to rows).


**Exercise:** As a test, create a new column titled 'testcolumn' with a blank value ('') and then remove it. 

In [ ]:
solution(Q2)

## Joining and Appending Datasets

Often we will find ourselves with two related tables that we want to combine in order to analyze them together. We might have two tables with **different variables for the same observations**; in this case, we can **join** them. Joining two tables results in a table that adds variables (columns) to existing records. 

> Join: A join combines rows in one or more tables based on common values

Alternatively, we might have two tables with the **same variables, and different observations**. In this case, we might want to add rows at the end of one table. For this, we **append** information to a DataFrame.

> Appending adds rows with like columns to a table.

### Joining Tables

We can join two dataframes based on a common key: Python will identify rows in both tables with the same key and output a new table with variables from both tables for that key.

There are 4 types of joins. Suppose we have two tables, left and right:

|Types of Joins ||
--------------|-----|
|Left| Keeps all rows in the left table, and will join these to rows in right table with a matching key.|
|Inner| Only gives us rows with matching in both tables (will also drop Left-only rows)|
|Outer| Returns all rows from both tables, regardless of if they have a match.|
|Right| Returns all rows in righ table, joined to any rows with a matching key from the left table.|

Be conscientious about the type of join you employ, as it will determine  the table(s) from which you might lose records.

The method for joining dataframes is ```df.merge()```, and it is specified in this way:

```python
df3 = df1.merge(df2, how='left', on='state')
```
The 'how' keyword specifies the type of join to execute, where df1 acts as the left table, and df2 acts as the right. 

The keyword 'on' determines the name of the common key. If the names of the key columns are not the same in both tables, you can use left_on and right_on to specify.

Let's import some information to join to our objects_table.

In [ ]:
people_table = pd.read_csv('Other_Files/NGAData/constituents.csv')
join_table = pd.read_csv('Other_Files/NGAData/objects_constituents.csv')

These tables are from a relational database, and objects_table and people_table are intended to be merged with the join_table. The join table consists of rows with  identifiers from both tables, as well as some attributes that describe the relationship.

So to relate 'objects' to 'people', we will have to realize two joins. Let's run the first one:

In [ ]:
join1 = objects_table.merge(join_table, how='left', on='objectid')

In [ ]:
join1.columns

As you can see if we look at the columns of the joined dataframe, rows now have incorporated information from the second table ('constituentid', 'displayorder','roletype', etc.) You might suspect what the key that could link this table to our people table might be (hint: the CSV that held the table was called 'constituents').

**Exercise 3:** Edit the code cell below to perform the second join by linking our first joined table (join1) to the people_table using a 'left' join and the key called 'constituentid'. 

If you want to check your answer in a new cell by calling join2, note that the object information should appear first, followed by the relationship, then the person (otherwise, you might have switched the order of the tables). 

In [ ]:
join2 = ____._____(______,how='____', on='______')

In [ ]:
join2.columns

In [ ]:
solution(Q3)

Now, if we make selections from the new table, we can make selections that relate the contents of both tables.

For instance, we can check the museum's holdings of works by the artist with ID 1400:

In [ ]:
join2[join2['constituentid']==1400][['title','preferreddisplayname']]

### Appending Datasets

To append datasets together, that is, to add one table as new rows in another, you need two tables with the same column structure.

Then, you use the ```df.append()``` method:

```python
df1.append(df2)
```

This method also accepts dictionaries and other series, in lieu of a second dataframe. For more information, consult the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html).

### Removing duplicate values

Another useful method, available for dataframes or series, is the option of finding only unique values.

For the full dataframe, this is is done with the df.drop_values() method.

```python
df2 = df.drop_values('columnname')
```
This will return a selection of rows without repeating the values in the column.

The series.unique() method will return a list of unique values within a single column:

```python
df['column_name'].unique()
```

For instance, we can generate a list of artists represented in the NGA collection, and count them:

In [ ]:
artists = people_table[people_table['artistofngaobject']==1]['preferreddisplayname']
artists = artists.unique()
artists = artists.tolist()
len(artists)

# Math on DataFrames

Series and DataFrames have mathematical methods associated to them like ```sum()```,```mean()```, ```median()```,```max()```, ```min()```...

In Pandas, math is applied efficiently to columns or rows. It is applied down columns by default. To act upon rows, you must specify otherwise by including axis=1 inside the parentheses.

Strings are ignored or handled in a logical way. NaN/Null values are ignored by default, instead of causing errors.

Dataframes have mathematical methods: 

```python
df.sum()
```

In [ ]:
print(objects_table.mean())

As do single columns, because they are series.

```
df['column'].sum()
```


In [ ]:
print(objects_table['beginyear'].mean())

In [ ]:
# formatting the above two examples to be legible?

You can also use operators to act upon values in columns. For instance:
    
```python
df['newcolumn'] = df['column1'] + df['column2']
```
Which returns a series of answers (adds cells in same row).

In [ ]:
df.transpose()
df.to_dict()
df.to_numpy()
df.to_json()
df.to_csv()

groupby

# Basic Plotting with Pandas

# Plotting functions

Dataframes also come with the plot() method, which allows us to create different types of graphs. 

There are two ways to access the plot method, either including the graph type in parenthesis or including it with a dot:

```python
df.plot(kind='line', x='column', y=['column1', 'column2'])
df.plot.line(x='column',y=['column1', 'column2'])
```

Plotting in Pandas is like plotting in excel: one column will be X values, all others will be Y. (Wide format, vs Tableau, Altair, Seaborn which require tall/tidy data: year in one column, field in another and one type of data per column. 

If we specify X in pandas, it will by default take all other columns for Y.

; gets rid of text output before plot.


# Bar Chart

df.mean() (of each column)

df.mean().plot.bar() vertical
df.mean().plot.barh() horizontal

Sorted bar charts
vertical bar charts go left to right, horizontal from down to up

df.mean().sort_values(ascending=False).plot.bar();

df.mean().sort_values().plot.barh();

# Histogram

df.hist() gives you a grid of all your different columns and a histogram of each of those. 10 bins by default. Shows you all range by default, not consistent x-y values through graphs. There is an argument for same axis on all

df.hist(sharex=True, sharey=True, layout(2,3), figsize=(10,5));

Another way to do it is
df.plot.hist()
Gives you something different: one set of axes and bins, different colors for multiple columns. you can set the alpha value down to make transparent. 

# Box Plot

df.plot.box()

Sort works the same, but took median separately, sorted it and then used their index to create a sorted list of column names.

horizontal box plot
df.plot.box(vert=false) -- different from bar charts

Syntax of plots are all slightly different, so powerful but there are better modules. 

---
Lesson Summary

<div style="text-align:center">    
  <a href="09%20Pandas.ipynb">Previous Lesson: Pandas</a>|
   <a href="11%20Power%20Up%20Your%20Python.ipynb">Next Lesson: Power up your Python</a>
</div>